## Essential Imports

In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

## Creating a new model

In [ ]:
model = Sequential()
model.add(MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               6422656   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 9,651,649
Trainable params: 6,422,785
Non-trainable params: 3,228,864
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-Mobilenet-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-Mobilenet-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 876s 3s/step - loss: 2.0920 - accuracy: 0.7544 - mean_absolute_error: 4.5518 - mean_squared_error: 129.4389 - hinge: 2.0741 - binary_crossentropy: 3.2433 - val_loss: 0.2383 - val_accuracy: 0.8964 - val_mean_absolute_error: 3.2429 - val_mean_squared_error: 15.1561 - val_hinge: 0.2248 - val_binary_crossentropy: 1.2373
Epoch 2/20
300/300 [==============================] - 23s 77ms/step - loss: 0.1333 - accuracy: 0.9500 - mean_absolute_error: 3.3180 - mean_squared_error: 15.6370 - hinge: 0.1205 - binary_crossentropy: 0.5770 - val_loss: 0.1430 - val_accuracy: 0.9336 - val_mean_absolute_error: 3.6566 - val_mean_squared_error: 18.6621 - val_hinge: 0.1321 - val_binary_crossentropy: 0.6348
Epoch 3/20
300/300 [==============================] - 23s 77ms/step - loss: 0.1154 - accuracy: 0.9599 - mean_absolute_error: 4.0240 - mean_squared_error: 22.4082 - hinge: 0.1050 - binary_crossentropy: 0.4459 - val_loss: 0.1584 - val_accuracy: 0.9364 - val_